In [ ]:
import csv
import os
import re
import glob
import time
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

In [ ]:
# Get the current working directory
py_folder_dir = os.getcwd()
parent_dir = os.path.dirname(py_folder_dir)
csv_file_path = os.path.join(parent_dir,"NZGD_sortedPairwiseDistances.csv")
csv_file_path

In [ ]:
def loginAndChangeDownloadDirectory(username, password, download_directory):
    
    # Create a new instance of the Chrome driver with the specified options
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_directory,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
    })
    driver = webdriver.Chrome(options=chrome_options)

    # URL of the login page
    login_url = "https://www.nzgd.org.nz/Registration/Login.aspx#t3"

    # Navigate to the login page
    driver.get(login_url)

    # Find the username and password input fields and fill them in
    driver.find_element(By.ID, "UserName").send_keys(username)
    driver.find_element(By.ID, "Password").send_keys(password)

    # Submit the login form
    driver.find_element(By.NAME, "ctl00$MainContent$LoginControl$LoginBox$LoginButton").click()

    return driver

In [ ]:
# Your login credentials
username = "your\username"
password = "your\password"

In [ ]:
df_cluster = pd.read_excel("\your\directory")
# for _,cluster_ in df_cluster.iterrows():
#     print(cluster_['ClusterName'])
df_cluster.iloc[390:]

In [ ]:
base_download_directory = os.path.abspath(os.path.join(py_folder_dir, os.pardir))
targetFolder="NZGD all data"
download_directory = os.path.join(base_download_directory, targetFolder)
# df_cluster = pd.read_excel("D:\MATLAB_DRIVE\MATLAB_PROJ\Xu\CPT数据库 (1)\cluster_ranking.xlsx")
if not os.path.exists(download_directory):
    os.mkdir(download_directory)

df_cluster = pd.read_excel("your\directory")
df_cluster = df_cluster.iloc[390:]
# for _,cluster_ in df_cluster.iterrows():
#     print(cluster_['ClusterName'])

for _,cluster_ in df_cluster.iterrows():
    cluster = cluster_['ClusterName']
    print(cluster)
    driver = loginAndChangeDownloadDirectory(username=username, password=password, download_directory=download_directory)
    # Read CSV file
    # csv_file_path = os.path.join(parent_dir,"NZGD_sortedPairwiseDistance.csv")
    csv_file_path = os.path.join(f"\your\directory") # 19137
    url_file_path = os.path.join(parent_dir,"NZGD_all_CPTs_20231127_converted.csv")
    output_log_path = 'log.txt' 

    # Read CSV file using pandas
    df = pd.read_csv(csv_file_path, encoding='utf-8')
    df_url = pd.read_csv(url_file_path, encoding='utf-8')

    # IDs = pd.concat([df["p1"], df["p2"]]).unique()
    IDs = df["ID"]
    df_url = df_url[df_url["ID"].isin(IDs)]#.iloc[2500:]


    with open(output_log_path, 'w') as log_file:
        count = 1
        for index, row in df_url.iterrows():
            print(f"Downloading {count}, {row['ID']}")

            count += 1
            
            target_files = glob.glob(os.path.join(download_directory, row['ID']+'_*'))
            if not target_files:
                # Wait for the table to load
                # Build target URL for each row
                target_url = row['URL']
                driver.get(target_url)
                # Navigate to the target URL
                try:
                    element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, "//table[@class='table-condensed']/tbody[@ng-repeat='f in investigationLog.files']"))
                    )
                except Exception as e:
                    pass
                # Get the page source
                html_source = driver.page_source
                # Parse the HTML content of the page using BeautifulSoup
                soup = BeautifulSoup(html_source, 'html.parser')
                # Find all rows in the table
                rows = soup.find_all("td", class_="text-right ng-binding", text=["Raw data", "Graphical log", "AGS4"])

                if len(rows) == 0:
                    log_file.write(f"Error: Raw data link not found for row {row['ID']}\n")
                    # pass
                else:
                    for r in rows:
                        # Extract file type from the text of the first <td> element
                        raw_fileName_node = r.find_next("a")
                        # file_type = Path(raw_fileName_node.text.lower()).suffix[1:]
                        # Extract the download link
                        download_link = r.find_next("a")["href"]
                        # Build the full URL using JavaScript and open it in a new tab
                        driver.execute_script(f"window.open('{download_link}', '_blank');")
            else:
                print(f"{row['ID']} already exists")
    time.sleep(30)
    driver.quit()


In [ ]:
csv_file_path = os.path.join(parent_dir,"NZGD_sortedPairwiseDistance.csv")
df = pd.read_csv(csv_file_path, encoding='utf-8')
dfa = df["p1"].unique()
IDs = pd.concat([df["p1"], df["p2"]]).unique()
dfa.size
url_file_path = os.path.join(parent_dir,"NZGD_all_CPTs_20231127_converted.csv")
df_url = pd.read_csv(url_file_path)
len(df_url[df_url['ID'].isin(IDs)])
